## Recommendation Tests to Count accuracy and other stuffs

In [18]:
# Import django for jupyter file to import other stuff
from django_for_jupyter import init_django
init_django("VehicleGenie_V1")

In [19]:
# Importing necessary models and modules
from app.models import vehicles, UserVehicle

from sklearn.metrics.pairwise import cosine_similarity
from django_pandas.io import read_frame
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as panda
import numpy as np

#### Setup the variables

### Import datasets from db and convert into data frames

In [20]:
# Import user vehicle db table as a query set
userVehicle = UserVehicle.objects.all()
vehicleQS = vehicles.objects.all()

In [21]:
# Read the query set to df
uvDf = read_frame(userVehicle)
vDf = read_frame(vehicleQS)
vDf = vDf.rename(columns={'vehicle_Id': 'Vehicle_Id'})

OperationalError: (2006, 'Server has gone away')

In [ ]:
# Join the dataframes on Vehicle_Id column
joined_df = panda.merge(uvDf, vDf, how='inner', on='Vehicle_Id')

# Select all columns from both dataframes
selected_df = joined_df.loc[:, ~joined_df.columns.duplicated()]

# Print the selected dataframe
selected_df.head()

In [ ]:
uvUpdatedDF = selected_df.loc[:,['Client_Id', 'Vehicle_Id', 'brand', 'model', 'transmission', 'body', 'year', 'fuel', 'location']]
uvUpdatedDF.head()

### Creating Pivot table to make a User Vehicle Matrix

In [ ]:
pivot_table = panda.pivot_table(selected_df, values='Vehicle_Id', index=['Client_Id'], columns=['brand', 'model', 'transmission', 'body', 'year', 'fuel', 'location'], aggfunc='count', fill_value=0)
pivot_table

### Get Recommendations

In [ ]:
clientId = 2

In [ ]:
usersBrowsingData = pivot_table.loc[clientId]
usersBrowsingData

In [ ]:
usersBrowsingData = user_history = np.array(usersBrowsingData).reshape(1, -1)
usersBrowsingData.ndim

In [ ]:
cosineSimilarity = cosine_similarity(pivot_table, usersBrowsingData)
cosineSimilarity = cosineSimilarity.flatten()
cosineSimilarity

In [ ]:
topMostVehicles = np.argsort(cosineSimilarity)[::-1][:10]
topMostVehicles